In [1]:
import tensorflow as tf

In [2]:
import os
import numpy 
import cv2
import hashlib
import cv2
import sys

In [3]:
from object_detection.utils import dataset_util

### Sample example

In [7]:
f = open('./wider_face_split/wider_face_train_bbx_gt.txt')
imgpath = f.readline().rstrip()
imgpath = os.path.join('./WIDER_train/images/', imgpath)
no_of_faces = f.readline().rstrip()
image = cv2.imread(imgpath)
for i in no_of_faces:
    details = f.readline().rstrip().split()
    details = list(map(int, details))
    print(details)
    cv2.rectangle(image, (details[0], details[1]), (details[0]+details[2], details[1]+details[3]), (255,0,0), 2)
cv2.imshow("img", image); cv2.waitKey(0); cv2.destroyAllWindows()
f.close()

[449, 330, 122, 149, 0, 0, 0, 0, 0, 0]


## Overall Analysis

Checks to find out the following:
1. Whether all images from annotation files are actually present or not
2. The number of images with zero faces

In [5]:
f = open('./wider_face_split/wider_face_train_bbx_gt.txt')
count1 = 0
count2 = 0
count3 = 0
no_of_faces= 0
while True:
    line = f.readline().rstrip()
    if not line:
        break
    elif '.jpg' in line:
        if( os.path.isfile(os.path.join('./WIDER_train/images/', line)) ):
            count2 += 1
        this_no_of_faces = int (f.readline().rstrip())
        no_of_faces += this_no_of_faces
        count1 = count1 + 1
        if (this_no_of_faces == 0):
            count3+=0
f.close()        
print ('Number of images written in annotation file: ',count1)
print('Number of images actually found: ', count2)
print('Total number of faces: ', no_of_faces)
if count1 == count2:
    print('All images from annotations were found in dataset')
else:
    print('Mismatch present')
print('Number of images with Zero faces: ', count3)

Number of images written in annotation file:  12880
Number of images actually found:  12880
Total number of faces:  159420
All images from annotations were found in dataset
Number of images with Zero faces:  0


### Bounding Box Integrity Check

This test is to check whether the face bounding boxes lie outside image dimmensions

In [65]:
f = open('./wider_face_split/wider_face_train_bbx_gt.txt')
count1 = 0
count2=0
no_of_faces= 0

while True:
    line = f.readline().rstrip()
    if not line:
        break
    elif '.jpg' in line:
        no_of_faces = 0
        filepath = os.path.join('./WIDER_train/images/', line)
        
        image = cv2.imread(filepath)
        height, width, channel = image.shape        
        no_of_faces = int (f.readline().rstrip())
        for i in range(no_of_faces):
            face_desc = f.readline().rstrip().split()
            face_desc = list(map(int, face_desc))
            xmin = face_desc[0]
            ymin = face_desc[1]
            xmax = xmin + face_desc[2]
            ymax = ymin + face_desc[3]
            if ( (xmax > width) & (ymax > height) ):
                count1 += 1
            else:
                count2 += 1
            
f.close()        
print ('Number of faceboxes outside image: ',count1)

Number of faceboxes outside image:  0


## Create TF Example and TFRecord

In [4]:
output_path = os.path.join('./output/', 'train.tfrecord')
output_path

'./output/train.tfrecord'

In [10]:
def print_progress(count, total):
    pct_complete = float(count) / total
    msg = "\r- Progress: {0:.1%}  ".format(pct_complete)
    sys.stdout.write(msg)
    sys.stdout.flush()

In [7]:
def create_tfexample(jpgline,no_of_faces, f):
    
    filepath = os.path.join('./WIDER_train/images/',jpgline)
    filename = jpgline.split('/')[1].encode('utf-8')
    
    image = cv2.imread(filepath)
    height, width, channel = image.shape
    
    encoded_image_data = open(filepath, 'rb').read()
    key = hashlib.sha256(encoded_image_data).hexdigest()
    
    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    poses = []
    truncated = []
    difficult_obj = []
    pose = ''
    
    for x in range(no_of_faces):

        details_array = f.readline().rstrip().split()
        details_array = list(map(int, details_array))

        xmins.append( float(details_array[0]/width) )
        xmaxs.append( float((details_array[0]+details_array[2])/width) )
        ymins.append( float(details_array[1]/height) )
        ymaxs.append( float((details_array[1]+details_array[3])/height) )
        classes_text.append(b'face')
        classes.append(1)
        pose = 'typical' if (int(details_array[-1]) == 0) else 'atypial'
        poses.append(pose.encode('utf8'))
        truncated.append(int(0))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(int(height)),
        'image/width': dataset_util.int64_feature(int(width)),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/difficult': dataset_util.int64_list_feature(int(0)),
        'image/object/truncated': dataset_util.int64_list_feature(truncated),
        'image/object/view': dataset_util.bytes_list_feature(poses),
        }))

    return tf_example       

In [11]:
f = open('./wider_face_split/wider_face_train_bbx_gt.txt')
writer = tf.io.TFRecordWriter(output_path)
count = 0
while True:
    line = f.readline().rstrip()
    if not line:
        break
    elif '.jpg' in line:
        no_of_faces = int(f.readline().rstrip())
        if(no_of_faces > 0):
            tf_example = create_tfexample(line, no_of_faces, f)
            writer.write(tf_example.SerializeToString())
        count+=1
        #print(count)
        print_progress(count=count, total=12880)
print('No. of images processed: ', count)            
writer.close()                 
f.close()

- Progress: 100.0%  No. of images processed:  12880
